Idea:

- Each number is independent. 
- Hence, the total repetitions for each num will likely be equal in the long term.
- Find the relations between a number to the rest within each draw. 
  - For example, if number 1 is drawn, what are the chances for the other particular number to be drawn based on the previous draws.
  - So each number will have a dictionary of other 44 numbers while each nested dictionary has two keys: Winning and Supps.
Later, give more weights to those have higher chances when generating a random ticket number.


The process:
1. Scan the previous draws and list the dictionary for the number relations.
2. Setting some threshold and weights to generate a random ticket
3. Each play has 10 tickets
4. Check the results: Hypothesis tests


In [298]:
# Library Imports
import csv
import random
import pandas as pd
from enum import Enum

In [307]:
# Previous Lotto Records into Lists
wedLotto = []
satLotto = []
with open('Wednesday Lotto Results - G NETWORK.csv', 'r') as file:
    csv_reader = csv.reader(file, delimiter=',')
    for row in csv_reader:
        wedLotto.append(row)

with open('Saturday Lotto Results - G NETWORK.csv', 'r') as file:
    csv_reader = csv.reader(file, delimiter=',')
    for row in csv_reader:
        satLotto.append(row)
        
wedLotto = wedLotto[1::]
satLotto = satLotto[1::]

In [308]:
# Go through the previous draws and mark the relations between numbers
wedRelations = {}
satRelations = {}

# list, set, dict are mutable -> pass by reference (if not reassign)
# ig. relationSetter(wedRelations, wedLotto)
def relationSetter(relations, prev_draws) -> dict:
    # String List to Int List
    for draw in prev_draws:
        try:
            drawnTicket = list(map(int, draw[1::]))
        except:
            continue
        for myIndex in range (0, len(drawnTicket)):
            num = drawnTicket[myIndex]
            for itsIndex in range (0, len(drawnTicket)):
            #for num in drawnTicket:
                itsSupp = True if itsIndex > 5 else False
                numNum = drawnTicket[itsIndex]
                if num != numNum: # Not repeating myself
                    if not num in relations:
                        relations[num] = {}
                    if not numNum in relations[num]:
                        relations[num][numNum] = 0
                    else:
                        relations[num][numNum] += 1

In [319]:
# Get relation between the numbers
relationSetter(wedRelations, wedLotto)
relationSetter(satRelations, satLotto)

In [320]:
df_wed = pd.DataFrame.from_dict(wedRelations)
df_sat = pd.DataFrame.from_dict(satRelations)

In [332]:
# Get best pals (numbers) to be for a particular number
def closePalsGenerator(tableX, numToGenerate) -> dict:
    sorted_table = tableX.sort_values(ascending= False)
    retDict = {}
    table_mean = sorted_table.mean()
    num_pals = 2*numToGenerate
    for key, value in sorted_table[::num_pals].items():
        retDict[key] = value
    
    retDict['mean'] = round(table_mean, 2)
    return retDict
    
# df_<>_closePals : include num_close_pals with its overall mean val
df_wed_closePals = {}
df_sat_closePals = {}
num_close_pals = 4
for num in range (1,46):
    df_wed_closePals[num] = closePalsGenerator(df_wed.get(num), num_close_pals)

for num in range (1,46):
    df_sat_closePals[num] = closePalsGenerator(df_sat.get(num), num_close_pals)
   

In [333]:
df_sat_closePals

{1: {15: 59.0, 36: 47.0, 12: 43.0, 31: 41.0, 8: 35.0, 35: 29.0, 'mean': 40.05},
 2: {21: 47.0,
  23: 31.0,
  25: 29.0,
  22: 25.0,
  32: 21.0,
  27: 15.0,
  'mean': 26.36},
 3: {38: 51.0,
  17: 35.0,
  41: 29.0,
  30: 27.0,
  45: 25.0,
  14: 21.0,
  'mean': 28.27},
 4: {27: 45.0, 18: 39.0, 43: 31.0, 15: 29.0, 28: 27.0, 35: 19.0, 'mean': 30.5},
 5: {14: 43.0,
  30: 39.0,
  35: 33.0,
  24: 31.0,
  40: 27.0,
  19: 23.0,
  'mean': 31.77},
 6: {31: 47.0, 29: 33.0, 1: 31.0, 12: 29.0, 38: 27.0, 24: 21.0, 'mean': 29.23},
 7: {20: 53.0, 14: 37.0, 5: 33.0, 29: 29.0, 12: 25.0, 42: 21.0, 'mean': 31.14},
 8: {23: 45.0, 1: 35.0, 4: 29.0, 41: 27.0, 32: 23.0, 19: 19.0, 'mean': 28.27},
 9: {33: 43.0, 13: 35.0, 27: 31.0, 21: 27.0, 25: 25.0, 3: 19.0, 'mean': 28.27},
 10: {1: 53.0, 32: 41.0, 15: 37.0, 3: 35.0, 8: 31.0, 35: 27.0, 'mean': 35.27},
 11: {15: 55.0, 21: 43.0, 5: 39.0, 10: 37.0, 34: 33.0, 2: 27.0, 'mean': 37.18},
 12: {31: 47.0,
  45: 37.0,
  5: 33.0,
  15: 31.0,
  17: 25.0,
  19: 21.0,
  'mean'

In [ ]:
# Generate combinations
# 1. randomly generate a starting number
# 2. Choose the next number with threshold
# threshold: percentage to its mean value?
# 3. If threshold low -> random?
# ticketGenerator(df_sat_closePals, anyNum, anyNum)
def ticketGenerator(closePals, startingNum, threshold) -> dict:
    retDict = {}
    retDict['ticket'] = [startingNum]
    retDict['mean'] = 0
    
    for index in range (0,7):
        nextNum = 
    
    

In [ ]:
# enum for diff lotto
class Lotto(Enum):
    WED = 1
    SAT = 2
    
class Division(Enum):
    first = "First"
    second = "Second"
    third = "Third"
    fourth = "Fourth"
    fifth = "Fifth"
    sixth = "Sixth"
    failed = "Failed"
    

def ticketMarker(tickets, lotto, drawnNums) -> dict:
    winning_match = 0
    supp_match = 0
    # String List to Int List
    drawnTicket = list(map(int, wedLotto[1][1::]))
    results = {'First':0, 'Second':0, 'Third':0, 'Fourth':0,
              'Fifth':0, 'Sixth':0, 'Failed':0}
    for ticket in tickets:
        print(ticket)
        results[divisionCalc(ticket, lotto, drawnTicket)] += 1
    
    return results

    

def divisionCalc(ticket, lotto, drawnTicket) -> str:
    winning_match = 0
    supp_match = 0
    
    # To compare two lists -> need two indices
    drawnIndex = 0
    ticketIndex = 0
    while ticketIndex < len(ticket) and drawnIndex < len(ticket):
        ticketNum = ticket[ticketIndex]
        drawnNum = drawnTicket[drawnIndex]
        if ticketNum < drawnNum:
            # missed the drawnNum - drawnNum holds
            ticketIndex += 1
            continue 
        elif ticketNum == drawnNum:
            if index > 5: 
                supp_match += 1
            else:
                winnings_match += 1
            # Hit the drawnNum; Move to the next one
            drawnIndex += 1; ticketIndex += 1;
        else:
            # smaller than drawnNums - ticketNum holds
            drawnIndex += 1
    
    # Division Check
    # Rules - Common
    # 1. 6 winnings 2. 5 winnings + 1 supp 3. 5 winnings
    # 4. 4 winnings 
    if winning_match == 6:
        return Division.first.value
    elif winning_match == 5:
        if supp_match == 1:
            return Division.second.value
        else:
            return Division.third.value
    elif winning_match == 4:
        return  Division.fourth.value
    
    # Rules - Wed
    # 5. 3 winnings + 2 supp 6. total 3 (winning 1 + 2, 2 + 1)
    # Rules - Sat
    # 5. 3 winnings + 1 supp 6. 3 winnings
    if lotto == Lotto.WED:
        if winning_match == 3 and supp_match == 2:
            return Division.fifth.value
        else:
            if winning_match + supp_match == 3:
                return Division.sixth.value
            else:
                return Division.failed.value
    else:
        if winning_match == 3 and supp_match == 2:
            return Division.fifth.value
        else:
            if winning_match == 3:
                return Division.sixth.value
            else:
                return Division.failed.value
        